In [ ]:
# importing important libraries to use

import tensorflow as tf
from tensorflow import keras
from keras.models import Model
#from keras.models import Sequential
#from keras.layers import BatchNormalization, Conv2D, Dropout, Dense 
#from keras.layers import GaussianNoise, GlobalAveragePooling2D, MaxPool2D
from keras.layers import Input, Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, Flatten
from keras.layers.merge import concatenate

from keras import losses
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
#from keras.utils import multi_gpu_model

from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from keras.layers.merge import concatenate

import random
import os
import pandas as pd
from statistics import mean, stdev
import sklearn
from sklearn.utils import shuffle

#from kfolds import build2LevelDataframe, makeSubdirFolds
#from kfolds import kfoldValidateModelPhotos, kfoldValidateModelSessions



########################################################################
#
# Downloads the MNIST data-set for recognizing hand-written digits.
#
# Implemented in Python 3.6
#
# Usage:
# 1) Create a new object instance: data = MNIST(data_dir="data/MNIST/")
#    This automatically downloads the files to the given dir.
# 2) Use the training-set as data.x_train, data.y_train and data.y_train_cls
# 3) Get random batches of training data using data.random_batch()
# 4) Use the test-set as data.x_test, data.y_test and data.y_test_cls
#
########################################################################
#
# This file is part of the TensorFlow Tutorials available at:
#
# https://github.com/Hvass-Labs/TensorFlow-Tutorials
#
# Published under the MIT License. See the file LICENSE for details.
#
# Copyright 2016-18 by Magnus Erik Hvass Pedersen
#
########################################################################

import numpy as np
import gzip
import os
import sys
import urllib.request
import tarfile
import zipfile
#from sklearn.preprocessing import OneHotEncoder
#from dataset import one_hot_encoded
#from download import download



In [ ]:
# Base URL for downloading the data-files from the internet.
base_url = "https://storage.googleapis.com/cvdf-datasets/mnist/"

# Filenames for the data-set.
filename_x_train = "train-images-idx3-ubyte.gz"
filename_y_train = "train-labels-idx1-ubyte.gz"
filename_x_test = "t10k-images-idx3-ubyte.gz"
filename_y_test = "t10k-labels-idx1-ubyte.gz"

In [ ]:
def _print_download_progress(count, block_size, total_size):
    """
    Function used for printing the download progress.
    Used as a call-back function in maybe_download_and_extract().
    """

    # Percentage completion.
    pct_complete = float(count * block_size) / total_size

    # Limit it because rounding errors may cause it to exceed 100%.
    pct_complete = min(1.0, pct_complete)

    # Status-message. Note the \r which means the line should overwrite itself.
    msg = "\r- Download progress: {0:.1%}".format(pct_complete)

    # Print it.
    sys.stdout.write(msg)
    sys.stdout.flush()


########################################################################
def download(base_url, filename, download_dir):
    """
    Download the given file if it does not already exist in the download_dir.
    :param base_url: The internet URL without the filename.
    :param filename: The filename that will be added to the base_url.
    :param download_dir: Local directory for storing the file.
    :return: Nothing.
    """

    # Path for local file.
    save_path = os.path.join(download_dir, filename)

    # Check if the file already exists, otherwise we need to download it now.
    if not os.path.exists(save_path):
        # Check if the download directory exists, otherwise create it.
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)

        print("Downloading", filename, "...")

        # Download the file from the internet.
        url = base_url + filename
        file_path, _ = urllib.request.urlretrieve(url=url,
                                                  filename=save_path,
                                                  reporthook=_print_download_progress)

        print(" Done!")

def one_hot_encoded(class_numbers, num_classes=None):
    """
    Generate the One-Hot encoded class-labels from an array of integers.
    For example, if class_number=2 and num_classes=4 then
    the one-hot encoded label is the float array: [0. 0. 1. 0.]
    :param class_numbers:
        Array of integers with class-numbers.
        Assume the integers are from zero to num_classes-1 inclusive.
    :param num_classes:
        Number of classes. If None then use max(class_numbers)+1.
    :return:
        2-dim array of shape: [len(class_numbers), num_classes]
    """

    # Find the number of classes if None is provided.
    # Assumes the lowest class-number is zero.
    if num_classes is None:
        num_classes = np.max(class_numbers) + 1

    return np.eye(num_classes, dtype=float)[class_numbers]

In [ ]:
class MNIST:
    """
    The MNIST data-set for recognizing hand-written digits.
    This automatically downloads the data-files if they do
    not already exist in the local data_dir.
    Note: Pixel-values are floats between 0.0 and 1.0.
    """

    # The images are 28 pixels in each dimension.
    img_size = 28

    # The images are stored in one-dimensional arrays of this length.
    img_size_flat = img_size * img_size

    # Tuple with height and width of images used to reshape arrays.
    img_shape = (img_size, img_size)

    # Number of colour channels for the images: 1 channel for gray-scale.
    num_channels = 1

    # Tuple with height, width and depth used to reshape arrays.
    # This is used for reshaping in Keras.
    img_shape_full = (img_size, img_size, num_channels)

    # Number of classes, one class for each of 10 digits.
    num_classes = 10

    def __init__(self, data_dir="data/MNIST/"):
        """
        Load the MNIST data-set. Automatically downloads the files
        if they do not already exist locally.
        :param data_dir: Base-directory for downloading files.
        """

        # Copy args to self.
        self.data_dir = data_dir

        # Number of images in each sub-set.
        self.num_train = 55000
        self.num_val = 5000
        self.num_test = 10000

        # Download / load the training-set.
        x_train = self._load_images(filename=filename_x_train)
        y_train_cls = self._load_cls(filename=filename_y_train)

        # Split the training-set into train / validation.
        # Pixel-values are converted from ints between 0 and 255
        # to floats between 0.0 and 1.0.
        self.x_train = x_train[0:self.num_train] / 255.0
        self.x_val = x_train[self.num_train:] / 255.0
        self.y_train_cls = y_train_cls[0:self.num_train]
        self.y_val_cls = y_train_cls[self.num_train:]

        # Download / load the test-set.
        self.x_test = self._load_images(filename=filename_x_test) / 255.0
        self.y_test_cls = self._load_cls(filename=filename_y_test)

        # Convert the class-numbers from bytes to ints as that is needed
        # some places in TensorFlow.
        self.y_train_cls = self.y_train_cls.astype(np.int)
        self.y_val_cls = self.y_val_cls.astype(np.int)
        self.y_test_cls = self.y_test_cls.astype(np.int)

        # Convert the integer class-numbers into one-hot encoded arrays.
        self.y_train = one_hot_encoded(class_numbers=self.y_train_cls,
                                       num_classes=self.num_classes)
        self.y_val = one_hot_encoded(class_numbers=self.y_val_cls,
                                     num_classes=self.num_classes)
        self.y_test = one_hot_encoded(class_numbers=self.y_test_cls,
                                      num_classes=self.num_classes)
        #self.y_train = OneHotEncoder(class_numbers=self.y_train_cls,
        #                               num_classes=self.num_classes)
        #self.y_val = OneHotEncoder(class_numbers=self.y_val_cls,
        #                             num_classes=self.num_classes)
        #self.y_test = OneHotEncoder(class_numbers=self.y_test_cls,
        #                              num_classes=self.num_classes)

    def _load_data(self, filename, offset):
        """
        Load the data in the given file. Automatically downloads the file
        if it does not already exist in the data_dir.
        :param filename: Name of the data-file.
        :param offset: Start offset in bytes when reading the data-file.
        :return: The data as a numpy array.
        """

        # Download the file from the internet if it does not exist locally.
        download(base_url=base_url, filename=filename, download_dir=self.data_dir)

        # Read the data-file.
        path = os.path.join(self.data_dir, filename)
        with gzip.open(path, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=offset)

        return data

    def _load_images(self, filename):
        """
        Load image-data from the given file.
        Automatically downloads the file if it does not exist locally.
        :param filename: Name of the data-file.
        :return: Numpy array.
        """

        # Read the data as one long array of bytes.
        data = self._load_data(filename=filename, offset=16)

        # Reshape to 2-dim array with shape (num_images, img_size_flat).
        images_flat = data.reshape(-1, self.img_size_flat)

        return images_flat

    def _load_cls(self, filename):
        """
        Load class-numbers from the given file.
        Automatically downloads the file if it does not exist locally.
        :param filename: Name of the data-file.
        :return: Numpy array.
        """
        return self._load_data(filename=filename, offset=8)

    def random_batch(self, batch_size=32):
        """
        Create a random batch of training-data.
        :param batch_size: Number of images in the batch.
        :return: 3 numpy arrays (x, y, y_cls)
        """

        # Create a random index into the training-set.
        idx = np.random.randint(low=0, high=self.num_train, size=batch_size)

        # Use the index to lookup random training-data.
        x_batch = self.x_train[idx]
        y_batch = self.y_train[idx]
        y_batch_cls = self.y_train_cls[idx]

        return x_batch, y_batch, y_batch_cls


########################################################################

In [ ]:
NUM_GPU = 5

if NUM_GPU != 1:
    strategy = tf.distribute.MirroredStrategy()
else:
    strategy = tf.distribute.get_strategy()

with strategy.scope():
    input_layer = Input(shape = (28*28,))
    FX = Flatten()(input_layer)

    X = Dense(1024, activation = 'relu')(FX)
    #CX = X
    CX = concatenate([X, FX], axis = -1)

    X = Dense(1024, activation = 'relu')(CX)
    #CX = X
    CX = concatenate([X, FX], axis = -1)

    X = Dense(1024, activation = 'relu')(CX)
    #CX = X
    CX = concatenate([X, FX], axis = -1)

    X = Dense(10, activation = 'softmax')(CX)
    
    model = Model(input_layer, X, name = 'Cascade')

    model.summary()
    
    model.compile(
        optimizer=Adam(lr=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy'])


data = MNIST(data_dir="/media/data2/MNIST")

In [ ]:
x_train = tf.keras.utils.normalize(data.x_train, axis=1)
x_test = tf.keras.utils.normalize(data.x_test, axis=1)

val_data = (data.x_val, data.y_val)

hist = model.fit(x_train, data.y_train, epochs=4, batch_size=512, validation_data=val_data) 

accuracy = hist.history['val_accuracy'][-1]
print()
print("Accuracy: {0:.2%}".format(accuracy))
print()

1.ADAPTIVE BOOSTING

In [ ]:
# ADAPTIVE BOOSTING
# using Adaptive boosting to optimise the structure and parameters of ANN
# using sklearn adaboost classifier
from sklearn.ensemble import AdaBoostClassifier

# adaboost instance
adaboostclf = AdaBoostClassifier(model, n_estimators=5000, algorithm="SAMME", learning_rate=0.2, random_state=42)

# training the model
print("model under training.....")

In [ ]:
# using a own version of mnist dataset with preffered tweaks
# errors experienced using the first version of mnist dataset, one used on the model above

def sort_by_target(mnist):
    reorder_train = np.array(sorted([(target, i) for i, target in enumerate(mnist.target[:60000])]))[:, 1]
    reorder_test = np.array(sorted([(target, i) for i, target in enumerate(mnist.target[60000:])]))[:, 1]
    mnist.data[:60000] = mnist.data[reorder_train]
    mnist.target[:60000] = mnist.target[reorder_train]
    mnist.data[60000:] = mnist.data[reorder_test + 60000]
    mnist.target[60000:] = mnist.target[reorder_test + 60000]

try:
    from sklearn.datasets import fetch_openml
    mnist = fetch_openml('mnist_784', version=1, cache=True)
    mnist.target = mnist.target.astype(np.int8) # fetch_openml() returns targets as strings
    sort_by_target(mnist) # fetch_openml() returns an unsorted dataset
except ImportError:
    from sklearn.datasets import fetch_mldata
    mnist = fetch_mldata('MNIST original')
mnist["data"], mnist["target"]

In [ ]:
X, y = mnist["data"], mnist["target"]
X.shape

In [ ]:
y.shape

In [ ]:
X_train, X_test, y_train, y_test = X[:30000], X[30000:], y[:30000], y[30000:]

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
# using sgd classifier instead of the model above because of the errors encountered
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train)

In [ ]:
# using Adaptive boosting to optimise the structure and parameters of ANN
# using sklearn adaboost classifier
from sklearn.ensemble import AdaBoostClassifier

# adaboost instance
adaboostclf = AdaBoostClassifier(sgd_clf, n_estimators=5000, algorithm="SAMME", learning_rate=0.2, random_state=42)

# training the model
print("model under training.....")

In [ ]:
# fitting adaboostclf to the dataset
# the training time is long, couple minutes
search = adaboostclf.fit(X_train,y_train)
# summarize result
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)

In [ ]:
y_pred = adaboostclf.predict(X_test)

In [ ]:
adaboostclf.score(X_train,y_train)

2.**RANDOM** SEARCH

In [ ]:
# RANDOM SEARCH
# using random search to optimise the structure and paramers of ANN

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from scipy.stats import uniform
from scipy.stats import loguniform

distributions = dict(C=uniform(loc=0, scale=4),penalty=['l2', 'l1'])

# define search space
space = dict()
space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
space['C'] = loguniform(1e-5, 100)
# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
clf = RandomizedSearchCV(sgd_clf, space,n_iter=500,scoring='accuracy',n_jobs=-1,cv=cv, random_state=0)
search = clf.fit(X_train, y_train)
# summarize result
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)